# AR(1)プロセスからランダムウォーク、そしてMCMCへ

このノートブックでは、時系列解析の基礎となる **AR(1)プロセス**（1次自己回帰モデル）を出発点とし、その係数 $\phi$ が 1 に近づくにつれて現れる **ランダムウォーク（酔歩）**、そしてそのランダムウォークを「探索」に応用した **メトロポリス・ヘイスティングス法 (MCMC)** までの流れを、数式とインタラクティブなシミュレーションで地続きに解説します。

---

## 1. AR(1) プロセスとは

AR(1) プロセス（AutoRegressive process of order 1）は、**「現在の値が、1つ前の値とノイズで決まる」** 最もシンプルな時系列モデルです。

$$
y_t = \phi y_{t-1} + \varepsilon_t, \quad \varepsilon_t \sim N(0, \sigma^2)
$$

ここで、
- $y_t$: 時刻 $t$ における値
- $\phi$: **自己回帰係数** (Autoregressive coefficient)
- $\varepsilon_t$: ホワイトノイズ（平均0、分散$\sigma^2$の正規分布に従う）

この $\phi$ の値によって、時系列の挙動（**定常性**）が劇的に変化します。

- **$|\phi| < 1$ (定常)**: 平均 0 の周りを振動し、長期的には安定します。
- **$|\phi| = 1$ (単位根・ランダムウォーク)**: **非定常**。平均へ回帰せず、どこまでも彷徨います。
- **$|\phi| > 1$ (発散)**: 指数関数的に値が大きくなります。

### 自己相関 (ACF) と 偏自己相関 (PACF)
ARモデルの特徴は、**コレログラム**（自己相関図）に現れます。

- **自己相関関数 (ACF)**: 時間差（ラグ）$k$ だけ離れたデータ間の相関。
    - AR(1) の場合、ACFは $\phi^k$ で減衰します（$\phi=0.8$なら $0.8, 0.64, 0.512...$）。
- **偏自己相関関数 (PACF)**: 間の時点の影響を取り除いた直接的な相関。
    - AR(1) の場合、**ラグ1 ( $\phi$ ) だけが有意な値を持ち、ラグ2以降はゼロになります**（これがARモデルの識別基準です）。

---


### 【実験】 $\phi$ を変えて挙動を確認しよう

スライダーで $\phi$ を動かして、以下の点を確認してください。
1. **$\phi = 0.5$**: ACFは速やかに減衰し、PACFはラグ1のみ立つ。
2. **$\phi = 0.9$**: ACFの減衰が遅くなる（記憶が長く残る）。
3. **$\phi = 1.0$**: **ランダムウォーク**。ACFはほとんど減衰せず、1付近に留まる（これを「単位根」と呼ぶ）。


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import ipywidgets as widgets
from IPython.display import display
import japanize_matplotlib

def simulate_ar1_interactive(phi=0.8, sigma=1.0, T=200):
    # シミュレーション
    np.random.seed(42)
    y = np.zeros(T)
    epsilon = np.random.normal(0, sigma, T)
    
    # 初期値
    if abs(phi) < 1:
        y[0] = epsilon[0] / np.sqrt(1 - phi**2)
    else:
        y[0] = 0
        
    for t in range(1, T):
        y[t] = phi * y[t-1] + epsilon[t]
        
    # プロット作成
    fig = plt.figure(figsize=(15, 5))
    
    # 1. 時系列プロット
    ax1 = fig.add_subplot(1, 3, 1)
    ax1.plot(y)
    ax1.set_title(rf"時系列 $y_t = {phi} y_{{t-1}} + \varepsilon$")
    ax1.set_xlabel("時刻 $t$")
    ax1.set_ylabel("$y_t$")
    ax1.grid(True, alpha=0.3)
    if abs(phi) >= 1:
        ax1.text(0.05, 0.9, "非定常 (Non-Stationary)", transform=ax1.transAxes, color="red", fontweight="bold")
    else:
        ax1.text(0.05, 0.9, "定常 (Stationary)", transform=ax1.transAxes, color="blue", fontweight="bold")

    # 2. ACF (自己相関)
    ax2 = fig.add_subplot(1, 3, 2)
    plot_acf(y, lags=20, ax=ax2, title="自己相関 (ACF)")
    ax2.grid(True, alpha=0.3)
    
    # 3. PACF (偏自己相関)
    ax3 = fig.add_subplot(1, 3, 3)
    try:
        plot_pacf(y, lags=20, ax=ax3, title="偏自己相関 (PACF)", method='ywm')
    except:
        ax3.text(0.5, 0.5, "計算エラー (非定常)", ha='center')
    ax3.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# インタラクティブ・ウィジェット
w_phi = widgets.FloatSlider(value=0.8, min=-1.1, max=1.1, step=0.1, description=r'係数 $\phi$:', continuous_update=False)
display(widgets.interactive(simulate_ar1_interactive, phi=w_phi, sigma=widgets.fixed(1.0), T=widgets.fixed(200)))


---

## 2.5 AR(1)の相関構造を可視化（相関行列と平均分散の倍率）

ここでは **AR(1)の相関が平均の分散をどの程度押し上げるか** を、
1) 相関行列のヒートマップ、2) 分散比の曲線で見える化します。

* **相関行列**：$\rho(|i-j|)=\alpha^{|i-j|}$ が対角線から帯状に減衰
* **分散比**：$\mathrm{Var}(ar y_T)$ が iid の平均分散 $\sigma_u^2/T$ に対して何倍か

---


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# AR(1)相関行列のヒートマップ
T = 10
alpha = 0.5
corr = np.fromfunction(lambda i, j: alpha ** np.abs(i - j), (T, T))

plt.figure(figsize=(5, 4))
plt.imshow(corr, cmap='viridis', vmin=0, vmax=1)
plt.colorbar(label=r'$\alpha^{|i-j|}$')
plt.title('AR(1) correlation matrix (T=10, alpha=0.5)')
plt.xlabel('j')
plt.ylabel('i')
plt.tight_layout()
plt.show()

# 平均分散の倍率 (iid平均との比)
def var_mean_ar1(T, alpha, sigma_u2=1.0):
    # Var(ar y_T) = (1/T^2) * (T*gamma0 + 2*sum_{h=1}^{T-1}(T-h) gamma_h)
    gamma0 = sigma_u2
    h = np.arange(1, T)
    gamma_h = sigma_u2 * (alpha ** h)
    return (1.0 / T**2) * (T * gamma0 + 2.0 * np.sum((T - h) * gamma_h))

T_grid = np.arange(2, 101)
alphas = [0.2, 0.5, 0.8]
ratio_curves = {}
for a in alphas:
    var_ar = np.array([var_mean_ar1(t, a, sigma_u2=1.0) for t in T_grid])
    var_iid = 1.0 / T_grid
    ratio_curves[a] = var_ar / var_iid

plt.figure(figsize=(6, 4))
for a in alphas:
    plt.plot(T_grid, ratio_curves[a], label=f'alpha={a}')
plt.axhline(1.0, color='tab:blue', ls='--', lw=1)
plt.axvline(10, color='tab:blue', ls='--', lw=1)
plt.title(r'標本平均の分散比: $\mathrm{Var}(\bar{y}_T) / (\sigma_u^2/T)$')
plt.xlabel('T (sample size)')
plt.ylabel('ratio')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()


### 3カラム比較：定数中心・時間変動中心・酔歩

| 観点 | 定数中心（$\mu$固定） | 時間変動中心（$\mu_t$） | 酔歩（ランダムウォーク） |
| --- | --- | --- | --- |
| 代表式 | $y_t=\mu+u_t$ | $x_t=\mu_t+R_t$（$\mu_t=T_t+S_t$ など） | $x_t=x_{t-1}+\varepsilon_t$ |
| “中心”の意味 | 一定の平均 $\mu$ | 動く平均 $\mu_t$（トレンド/季節） | 中心が固定されず漂う |
| 定常性 | $u_t$ が定常なら $y_t$ も定常 | $\mu_t$ が動くので通常非定常 | 非定常（分散も増える） |
| ACFの典型形 | 幾何減衰（$\phi^h$） | 季節スパイク＋全体高め | 多くのラグで高い相関が残る |
| 直感 | 同じ中心の周りで揺れる | 動く中心の周りで揺れる | 中心そのものがノイズで歩く |

---


---

## 2. 定常から非定常へ：ランダムウォーク（酔歩）

$\phi = 1$ のとき、式は以下のようになります。

$$
y_t = y_{t-1} + \varepsilon_t
$$

これは、**「昨日の位置からランダムに一歩進んだ場所が今日の位置」** というモデルです。これを **ランダムウォーク（酔歩）** と呼びます。またの名を「イカ（Squid）」の遊泳とも喩えられます（予測不可能な動き）。

### なぜこれが重要なのか？
ランダムウォークは、**「分散が時間とともに増大する」** という性質を持ちます。
$$
\mathrm{Var}(y_t) = t \sigma^2
$$
つまり、時間が経てば経つほど、**どこにいるか予測がつかなくなる（＝探索範囲が広がる）** ということです。

統計学（特にベイズ統計）では、この「どこへ行くかわからない＝空間を広く探索できる」という性質を**逆に利用**します。それが次に紹介する **MCMC（マルコフ連鎖モンテカルロ法）** です。


---

## 3. ランダムウォークの応用：メトロポリス・ヘイスティングス法

確率分布 $p(x)$ からサンプリングを行いたいとき、その分布が複雑だと直接サンプリングできません。そこで、**ランダムウォークを使って分布の「山」を探索**します。

ここから先は、MCMC を「探索」として理解するために、**探索だけど最適化ではない**という核心を丁寧に整理します。

---

### 3.1 MCMCを一言で：**地図の上を歩いて、よく居る場所ほど長く滞在する仕組み**

* 目標：ある分布 $\pi(x)$（「この空間ではここに居る確率が高い」）からサンプルを取りたい
* でも直接サンプルできない
* だから **歩き回る（Markov chain）** ことで、長期的に「居る頻度」が $\pi(x)$ に一致するように設計する

この “居る頻度” のことを **定常分布（stationary distribution）** と呼びます。
MCMCは「歩き方を工夫して、定常分布を欲しい $\pi$ に合わせる技術」です。

---

### 3.2 「探索」だけど、探索の意味が最適化と違う

#### 最適化（例：勾配降下）

* 目的：最大/最小の点（最良の点）を見つける
* “いい点”に集まり続け、最終的にそこへ吸い寄せられる

#### MCMC

* 目的：**点を当てるのではなく、分布を再現**する
* “いい点”に行くのは当然だが、そこに固定されてはダメ
* “そこそこ”の場所にも、$\pi(x)$ に見合う頻度で行く必要がある

つまり、MCMCの探索は

* 「山の頂点を当てる」探索ではなく
* 「山の形（面積）に比例して歩く」探索

---

### 3.3 何を探索してる？＝“状態空間（パラメータ空間）”

MCMCで探索する空間は状況によって違うけど、典型は2つです。

#### A) ベイズ推論（パラメータ空間）

* $x=\theta$（パラメータ）
* $\pi(\theta)=p(\theta\mid \text{data})$（事後分布）
* 「この $\theta$ がどれくらい尤もらしいか」の地形を歩く

#### B) 物理・確率モデル（状態空間）

* $x$ が粒子配置とか状態
* $\pi(x)\propto e^{-E(x)}$（エネルギーが低いほど高確率）

---

### 3.4 どうやって歩く？＝MH（メトロポリス・ヘイスティングス）を探索として言い直す

#### アルゴリズムの直感

1. **今いる場所** を $x_t$ とする
2. 次の候補 $x_{new}$ を、**ランダムウォーク** で決める
   $$ x_{new} = x_t + \varepsilon, \quad \varepsilon \sim N(0, \sigma^2) $$
   （これはまさに、$\phi=1$ の AR(1) プロセスの1ステップ）
3. $x_{new}$ が $x_t$ よりも「確率が高い（山頂に近い）」場所なら、**必ず移動**
4. 「確率が低い（麓の方）」場所なら、**一定確率で移動**

こうすることで、ランダムウォーク（酔っ払い）は、**確率が高い場所ほど頻繁に訪れる**ようになります。その足跡（Trace）を集めると、目的の確率分布に従うサンプルが得られます。

---

### 3.5 “探索の遅さ” とは何か：混合（mixing）と自己相関

MCMCの難所はここです。

* $\sigma$ が小さいと受理率は高い
  → でも一歩が小さすぎて、なかなか別の山へ行けない
  → 「同じ場所の近所をウロウロ」＝自己相関が強い
* $\sigma$ が大きいと遠くへ行ける
  → でも提案が無茶で棄却が増え、足踏みが多くなる

このバランスの良し悪しが **混合（mixing）** です。

---

### 3.6 探索の“成果”を測る物差し：$N_{\text{eff}}$（有効サンプルサイズ）

「探索ができてるか」は見た目のトレースだけじゃなく、数で測れます。

まず目的は「平均などの期待値」を推定することが多いです。

$$
I = E_\pi[f(X)]
$$

推定量：
$$
\hat I = \frac{1}{N}\sum_{t=1}^N f(x^{(t)})
$$

iidなら
$$
\mathrm{Var}(\hat I)\approx \frac{\sigma_f^2}{N}
$$

MCMCは相関があるので
$$
\mathrm{Var}(\hat I)\approx \frac{\sigma_f^2}{N}\Big(1+2\sum_{h\ge1}\rho_f(h)\Big)
$$

ここで
$$
\tau_{\text{int}}:=1+2\sum_{h\ge1}\rho_f(h)
$$

と置くと

$$
N_{\text{eff}}:=\frac{N}{\tau_{\text{int}}}
$$

* $\tau_{\text{int}}$ が大きい＝探索が遅い（相関が残る）
* $N_{\text{eff}}$ が小さい＝「独立に取れたのはこれだけ」という意味

---

### 3.7 「空間探索」としての直感をさらに強化する2つの可視化

#### A) “山の行き来”の可視化（モード指標）

混合正規で山が2つなら
$$
 g(x)=\mathbf{1}[x<3]
$$

みたいな指標を作ると、山の切替が見えます。

* 0/1が頻繁に切り替わる → 探索が広い
* ずっと0 or ずっと1 → 片山に閉じ込め

#### B) MSD（平均二乗変位）で“拡散の速さ”を見る

$$
\mathrm{MSD}(h)=E[(X_{t+h}-X_t)^2]
$$

* 小さい $\sigma$ だとMSDが伸びない（拡散が遅い）
* 適切な $\sigma$ だと伸びる（空間を移動できる）

「拡散の遅さ」は、まさにこのMSDで見えます。

---

### 3.8 まとめ：MCMCの“探索”を正確に言い直すとこう

* MCMCは **空間を探索する**
* ただし探索の目的は **最大点を探す** ではなく
* **確率質量に比例して滞在頻度を作る** こと
* 探索性能は **受理率より混合（相関の減衰）**
* それを測るのが $\tau_{\text{int}}$ と $N_{\text{eff}}$

---


### 目標分布の可視化
まずはサンプリングしたい **目標分布** $\pi(x)$ を図で確認します。


In [ ]:
from scipy.stats import norm

# 目標分布: 標準正規分布 N(0, 1)
def target_pdf(x):
    return norm.pdf(x, loc=0, scale=1)

x = np.linspace(-4, 4, 400)
plt.figure(figsize=(7, 4))
plt.plot(x, target_pdf(x), color='crimson', lw=3, label=r'$\pi(x)$（目標分布）')
plt.title('目標分布 $\pi(x)$', fontsize=13)
plt.xlabel('x')
plt.ylabel('密度')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()


### 提案分布の可視化（ランダムウォーク）
メトロポリス・ヘイスティングス法では、現在位置 $x_t$ から
$$x_{new} = x_t + ε,\quad ε \sim N(0, σ^2)$$
という **提案分布** を使います。ここでは $x_t=1.5$ と仮定して可視化します。


In [ ]:
current_x = 1.5
step_width = 1.0

proposal_x = np.linspace(-4, 6, 400)
proposal_pdf = norm.pdf(proposal_x, loc=current_x, scale=step_width)

plt.figure(figsize=(7, 4))
plt.plot(proposal_x, proposal_pdf, color='navy', lw=3, label='提案分布 $q(x\'|x_t)$')
plt.axvline(current_x, color='orange', ls='--', lw=2, label=r'現在位置 $x_t$')
plt.title('提案分布（ランダムウォーク）', fontsize=13)
plt.xlabel('x')
plt.ylabel('密度')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()


### 採択確率の形（途中計算の可視化）
採択確率は
$$
\alpha(x_{new}, x_t) = \min\left(1, \frac{\pi(x_{new})}{\pi(x_t)}\right)
$$
で与えられます。$x_t$ を固定したとき、$x_{new}$ に対する形を描いてみます。


In [ ]:
current_x = 1.5
x_new = np.linspace(-4, 4, 400)
ratio = target_pdf(x_new) / target_pdf(current_x)
accept_prob = np.minimum(1.0, ratio)

plt.figure(figsize=(7, 4))
plt.plot(x_new, accept_prob, color='purple', lw=3)
plt.axvline(current_x, color='orange', ls='--', lw=2, label=r'現在位置 $x_t$')
plt.title('採択確率 $\alpha(x_{new}, x_t)$', fontsize=13)
plt.xlabel('提案値 $x_{new}$')
plt.ylabel('採択確率')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()


---

## 3.3 提案分布 $q$ の対称性が“2回”出てくる理由

* **対称性①**：提案分布そのものの形（左右対称）
* **対称性②**：受理確率の式で $q(y|x)=q(x|y)$ が相殺される

一般形は
$$
\alpha(x,y)=\min\left(1,\frac{\pi(y)q(x|y)}{\pi(x)q(y|x)}\right)
$$
で、**対称提案**なら $q(x|y)=q(y|x)$ が成り立つため
$$
\alpha(x,y)=\min\left(1,\frac{\pi(y)}{\pi(x)}\right)
$$
に簡約されます。

---


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 対称な一様ランダムウォーク提案の可視化
x0 = 0.0
a = 1.0
x = np.linspace(-3, 3, 400)
q = np.where(np.abs(x - x0) <= a, 1/(2*a), 0.0)

plt.figure(figsize=(6, 3.5))
plt.plot(x, q, lw=2)
plt.title('Random-walk Uniform Proposal q(y|x) (x=0, a=1)')
plt.xlabel('y')
plt.ylabel('q(y|x=0)')
plt.tight_layout()
plt.show()

# 2Dヒートマップ（対角線対称を可視化）
grid = np.linspace(-3, 3, 200)
X, Y = np.meshgrid(grid, grid)
Q = np.where(np.abs(Y - X) <= a, 1/(2*a), 0.0)

plt.figure(figsize=(5, 4))
plt.imshow(Q, origin='lower', extent=[-3, 3, -3, 3], aspect='auto', cmap='viridis')
plt.title('Heatmap of q(y|x) (a=1) (diagonal symmetry: q(y|x)=q(x|y))')
plt.xlabel('x')
plt.ylabel('y')
plt.tight_layout()
plt.show()


---

## 3.4 境界で対称性が壊れる例（truncated / reject / reflect）

状態空間を $[0,10]$ に制限する場合、提案の扱い方で **対称性が壊れる** ことがあります。
ここでは 3つの方式を比較します。

* **truncated**：区間外を切って一様化（密度が非対称になる）
* **reject outside**：区間外ならその場に留まる
* **reflect**：区間外なら鏡で反射させる

---


---

### 3.4.1 境界処理のインタラクティブ可視化（a と方式を動かす）

* **方式**（truncated / reject / reflect）と **歩幅 $a$** を動かし、
  1Dスライス・ヒートマップ・非対称マップがどう変わるかをその場で確認します。

---


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# インタラクティブ用の境界提案分布
L, U = 0.0, 10.0
grid = np.linspace(L, U, 200)
X, Y = np.meshgrid(grid, grid)

def q_truncated(x, y, a):
    low = np.maximum(L, x - a)
    high = np.minimum(U, x + a)
    width = high - low
    return np.where((y >= low) & (y <= high), 1 / width, 0.0)

def q_reject(x, y, a):
    low = x - a
    high = x + a
    inside = (y >= L) & (y <= U) & (y >= low) & (y <= high)
    q = np.where(inside, 1 / (2 * a), 0.0)
    return q

def q_reflect(x, y, a):
    low = x - a
    high = x + a
    q = np.zeros_like(y)
    q += np.where((y >= max(L, low)) & (y <= min(U, high)), 1 / (2 * a), 0.0)
    y_reflect_L = 2 * L - y
    q += np.where((y_reflect_L >= low) & (y_reflect_L <= high), 1 / (2 * a), 0.0)
    y_reflect_U = 2 * U - y
    q += np.where((y_reflect_U >= low) & (y_reflect_U <= high), 1 / (2 * a), 0.0)
    return q

def render_boundary(mode='truncated', a=2.0, x_in=5.0, x_edge=0.5):
    if mode == 'truncated':
        q_func = q_truncated
        title_prefix = 'Truncated'
    elif mode == 'reject':
        q_func = q_reject
        title_prefix = 'Reject'
    else:
        q_func = q_reflect
        title_prefix = 'Reflect'

    y = np.linspace(L, U, 300)
    q_in = q_func(x_in, y, a)
    q_edge = q_func(x_edge, y, a)
    Q = q_func(X, Y, a)
    A = Q - q_func(Y, X, a)

    fig, axes = plt.subplots(1, 3, figsize=(14, 4))

    axes[0].plot(y, q_in, label=f'x={x_in:.2f} (interior)')
    axes[0].plot(y, q_edge, label=f'x={x_edge:.2f} (near boundary)')
    axes[0].set_title(f'{title_prefix}: q(y|x) slice')
    axes[0].set_xlabel('y')
    axes[0].set_ylabel('approx q(y|x)')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    axes[1].imshow(Q, origin='lower', extent=[L, U, L, U], aspect='auto', cmap='viridis')
    axes[1].set_title(f'{title_prefix}: heatmap q(y|x)')
    axes[1].set_xlabel('x')
    axes[1].set_ylabel('y')

    axes[2].imshow(A, origin='lower', extent=[L, U, L, U], aspect='auto', cmap='viridis')
    axes[2].set_title(f'{title_prefix}: asymmetry Q - Q^T')
    axes[2].set_xlabel('x')
    axes[2].set_ylabel('y')

    plt.tight_layout()
    plt.show()

mode_widget = widgets.ToggleButtons(options=['truncated', 'reject', 'reflect'], description='方式')
a_widget = widgets.FloatSlider(value=2.0, min=0.5, max=4.0, step=0.1, description='a', continuous_update=False)
x_in_widget = widgets.FloatSlider(value=5.0, min=L, max=U, step=0.5, description='x_in', continuous_update=False)
x_edge_widget = widgets.FloatSlider(value=0.5, min=L, max=U, step=0.1, description='x_edge', continuous_update=False)

ui = widgets.VBox([mode_widget, a_widget, x_in_widget, x_edge_widget])
out = widgets.interactive_output(render_boundary, {
    'mode': mode_widget,
    'a': a_widget,
    'x_in': x_in_widget,
    'x_edge': x_edge_widget,
})
display(ui, out)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 共通設定
L, U = 0.0, 10.0
a = 2.0
grid = np.linspace(L, U, 200)
X, Y = np.meshgrid(grid, grid)

def heatmap_q(q_func, title, a):
    Q = q_func(X, Y, a)
    plt.figure(figsize=(5, 4))
    plt.imshow(Q, origin='lower', extent=[L, U, L, U], aspect='auto', cmap='viridis')
    plt.title(title)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.tight_layout()
    plt.show()

def asymmetry_map(q_func, title, a):
    Q = q_func(X, Y, a)
    Q_t = q_func(Y, X, a)
    A = Q - Q_t
    plt.figure(figsize=(5, 4))
    plt.imshow(A, origin='lower', extent=[L, U, L, U], aspect='auto', cmap='viridis')
    plt.title(title)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.tight_layout()
    plt.show()

# 1Dスライス（内部と端付近の比較）
def plot_slice(q_func, title, a):
    x_in = 5.0
    x_edge = 0.5
    y = np.linspace(L, U, 300)
    q_in = q_func(x_in, y, a)
    q_edge = q_func(x_edge, y, a)
    plt.figure(figsize=(6, 3.5))
    plt.plot(y, q_in, label=f'x={x_in:.2f} (interior)')
    plt.plot(y, q_edge, label=f'x={x_edge:.2f} (near boundary)')
    plt.title(title)
    plt.xlabel('y')
    plt.ylabel('approx q(y|x)')
    plt.legend()
    plt.tight_layout()
    plt.show()

# truncated
plot_slice(q_truncated, 'Truncated Uniform: q(y|x) slice (approx density)', a)
heatmap_q(q_truncated, 'Heatmap of q(y|x) (truncated). Note: not symmetric about diagonal.', a)
asymmetry_map(q_truncated, 'Asymmetry map: q(y|x) - q(x|y) (truncated)', a)

# reject outside
plot_slice(q_reject, 'Reject outside: q(y|x) slice (approx density)', a)
heatmap_q(q_reject, 'Reject outside: Transition heatmap P(x->y-bin)', a)
asymmetry_map(q_reject, 'Reject outside: Asymmetry map P - P^T', a)

# reflect
plot_slice(q_reflect, 'Reflect: q(y|x) slice (approx density)', a)
heatmap_q(q_reflect, 'Reflect: Transition heatmap P(x->y-bin)', a)
asymmetry_map(q_reflect, 'Reflect: Asymmetry map P - P^T', a)


---

### 3.5 iid サンプル vs 酔歩MH（2カラム比較）

| 観点 | iid（$X_i\sim\pi$） | 酔歩MH（$X^{(t)}$ 連鎖） |
| --- | --- | --- |
| 固定される対象 | 目標分布 $\pi(x)$ | 目標分布 $\pi(x)$ |
| 生成される列 | 独立サンプル | 相関サンプル（連鎖） |
| 推定量 | $\hat I=rac{1}{N}\sum f(X_i)$ | $\hat I=rac{1}{N}\sum f(X^{(t)})$ |
| 分散 | $\sigma_f^2/N$ | $rac{\sigma_f^2}{N}(1+2\sum_{h\ge1}ho_f(h))$ |
| 有効サンプル | $N_{	ext{eff}}=N$ | $N_{	ext{eff}}=N/	au_{	ext{int}}$ |

---


In [ ]:

def run_metropolis_hastings(n_iter=5000, step_width=1.0):
    # 目標分布: 標準正規分布 N(0, 1) とあえて少しズラした N(3, 0.5) の混合分布を想定してみる
    # シンプルにするため、ここでは標準正規分布 N(0, 1) を目標とする
    np.random.seed(123)
    current_x = 10.0  # 初期値（あえて遠くからスタート）
    samples = []
    trace = []
    
    # 提案分布としてランダムウォークを使用
    # x_new = 1.0 * x_old + epsilon  (つまり phi=1 の AR(1))
    
    for i in range(n_iter):
        # 1. 提案 (Random Walk Step)
        epsilon = np.random.normal(0, step_width)
        proposal_x = current_x + epsilon
        
        # 2. 採択確率 (Metropolis基準)
        # p(x_new) / p(x_old)
        ratio = target_pdf(proposal_x) / target_pdf(current_x)
        acceptance_prob = min(1.0, ratio)
        
        # 3. 判定
        if np.random.rand() < acceptance_prob:
            current_x = proposal_x  # 採択（移動）
        else:
            current_x = current_x   # 棄却（滞留）
            
        trace.append(current_x)
        if i > 500: # バーンイン（初期の影響を除く）
            samples.append(current_x)
            
    # 可視化
    fig = plt.figure(figsize=(15, 6))
    
    # トレースプロット（時系列）
    ax1 = fig.add_subplot(1, 2, 1)
    ax1.plot(trace, lw=0.8, color='navy', alpha=0.7)
    # 生文字列でエスケープを回避
    ax1.set_title(rf"MCMCの軌跡 (トレースプロット)\nランダムウォーク $\phi=1$ による探索", fontsize=14)
    ax1.set_xlabel("Step $t$")
    ax1.set_ylabel("$x_t$")
    ax1.grid(True, alpha=0.3)
    ax1.text(0.05, 0.9, "探索的挙動 (探索)", transform=ax1.transAxes, color="green", fontweight="bold")

    # ヒストグラム（分布）
    ax2 = fig.add_subplot(1, 2, 2)
    ax2.hist(samples, bins=50, density=True, color='skyblue', alpha=0.7, label="MCMCサンプル")
    x = np.linspace(-4, 4, 100)
    ax2.plot(x, target_pdf(x), 'r-', lw=3, label="目標分布")
    ax2.set_title("サンプリング結果の分布", fontsize=14)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# 実行
run_metropolis_hastings()


---

## 4. 探索の可視化：モード指標とMSD

ここでは、**「山の行き来」** と **「拡散の速さ」** を見える化します。

* **モード指標**: $g(x)=\mathbf{1}[x<3]$ で山の切替を追跡
* **MSD**: $\mathrm{MSD}(h)=E[(X_{t+h}-X_t)^2]$ で拡散の速さを測定

---


In [ ]:

# 混合正規の目標分布

def target_mixture_pdf(x):
    return 0.5 * norm.pdf(x, loc=0, scale=1) + 0.5 * norm.pdf(x, loc=6, scale=1)


def run_mh_generic(n_iter=6000, step_width=1.0, target_pdf=target_mixture_pdf, seed=123):
    rng = np.random.default_rng(seed)
    current_x = 0.0
    trace = np.zeros(n_iter)
    for i in range(n_iter):
        proposal_x = current_x + rng.normal(0, step_width)
        ratio = target_pdf(proposal_x) / target_pdf(current_x)
        if rng.random() < min(1.0, ratio):
            current_x = proposal_x
        trace[i] = current_x
    return trace


def compute_msd(trace, max_lag=200):
    msd = [np.mean((trace[h:] - trace[:-h]) ** 2) for h in range(1, max_lag + 1)]
    return np.array(msd)


# 2つのステップ幅で比較
trace_small = run_mh_generic(step_width=0.5)
trace_large = run_mh_generic(step_width=3.0)

# モード指標 g(x)=1[x<3]
mode_small = (trace_small < 3).astype(int)
mode_large = (trace_large < 3).astype(int)

# MSD
msd_small = compute_msd(trace_small)
msd_large = compute_msd(trace_large)

# 可視化
fig = plt.figure(figsize=(12, 8))

# モード指標の時系列
ax1 = plt.subplot2grid((2, 2), (0, 0))
ax1.plot(mode_small[:400], lw=0.8)
ax1.set_title("モード指標 (小ステップ)")
ax1.set_ylim(-0.1, 1.1)
ax1.set_ylabel("g(x)")
ax1.grid(True, alpha=0.3)

ax2 = plt.subplot2grid((2, 2), (0, 1))
ax2.plot(mode_large[:400], lw=0.8, color='tab:orange')
ax2.set_title("モード指標 (大ステップ)")
ax2.set_ylim(-0.1, 1.1)
ax2.grid(True, alpha=0.3)

# MSD
ax3 = plt.subplot2grid((2, 2), (1, 0), colspan=2)
ax3.plot(msd_small, label='小ステップ', lw=2)
ax3.plot(msd_large, label='大ステップ', lw=2)
ax3.set_title("MSD (平均二乗変位)")
ax3.set_xlabel("ラグ h")
ax3.set_ylabel("MSD(h)")
ax3.legend()
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


---

### 4.1 インタラクティブ：歩幅 $a$ と系列長で混合を体感する

* **歩幅 $a$** と **反復数 $N$** を動かし、
  トレース／ヒスト／ACF／MSD がどう変わるかを同時に確認します。

---


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

def target_mixture_pdf(x):
    return 0.25 * norm.pdf(x, loc=0, scale=1) + 0.75 * norm.pdf(x, loc=6, scale=1)

def mh_trace(step_width, n_total, seed=42):
    rng = np.random.default_rng(seed)
    x = 0.0
    trace = np.zeros(n_total)
    acc = 0
    for i in range(n_total):
        proposal = x + rng.normal(0, step_width)
        ratio = target_mixture_pdf(proposal) / target_mixture_pdf(x)
        if rng.random() < min(1.0, ratio):
            x = proposal
            acc += 1
        trace[i] = x
    return trace, acc / n_total

def acf_simple(x, max_lag=100):
    x = np.asarray(x)
    x = x - x.mean()
    denom = np.dot(x, x)
    if denom == 0:
        return np.zeros(max_lag + 1)
    out = [1.0]
    for h in range(1, max_lag + 1):
        out.append(np.dot(x[:-h], x[h:]) / denom)
    return np.array(out)

def msd_simple(x, max_lag=100):
    x = np.asarray(x)
    out = []
    for h in range(1, max_lag + 1):
        d = x[h:] - x[:-h]
        out.append(np.mean(d * d))
    return np.array(out)

def render_mh_mix(a=1.0, n_total=5000, burn=500, seed=42):
    trace, acc = mh_trace(a, n_total, seed=seed)
    trace = trace[burn:]
    acf_x = acf_simple(trace, max_lag=80)
    msd_x = msd_simple(trace, max_lag=80)

    fig, axes = plt.subplots(2, 2, figsize=(12, 8))

    axes[0, 0].plot(trace, lw=0.7)
    axes[0, 0].set_title(f'トレース (a={a:.2f}, acc={acc:.2f})')
    axes[0, 0].set_xlabel('t')
    axes[0, 0].set_ylabel('x')
    axes[0, 0].grid(True, alpha=0.3)

    axes[0, 1].hist(trace, bins=50, density=True, alpha=0.6, label='MCMC')
    xs = np.linspace(-4, 10, 400)
    axes[0, 1].plot(xs, target_mixture_pdf(xs), lw=2, label='target')
    axes[0, 1].set_title('ヒストグラム')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)

    axes[1, 0].plot(acf_x)
    axes[1, 0].axhline(0, color='black', lw=1)
    axes[1, 0].set_title('ACF (x)')
    axes[1, 0].set_xlabel('lag')
    axes[1, 0].grid(True, alpha=0.3)

    axes[1, 1].plot(msd_x, color='tab:orange')
    axes[1, 1].set_title('MSD (x)')
    axes[1, 1].set_xlabel('lag')
    axes[1, 1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

a_widget = widgets.FloatLogSlider(value=1.0, base=10, min=-1, max=1, step=0.05, description='a')
n_widget = widgets.IntSlider(value=5000, min=1000, max=20000, step=1000, description='N')
burn_widget = widgets.IntSlider(value=500, min=0, max=5000, step=100, description='burn')
seed_widget = widgets.IntText(value=42, description='seed')

ui = widgets.VBox([a_widget, n_widget, burn_widget, seed_widget])
out = widgets.interactive_output(render_mh_mix, {
    'a': a_widget,
    'n_total': n_widget,
    'burn': burn_widget,
    'seed': seed_widget,
})
display(ui, out)


---

## 5. aを連続スイープして混合効率を可視化する

ここからは、**歩幅 $a$ を連続スイープ**して、探索の「成果」をまとめて確認します。

今回の設定（混合正規 $\tfrac{1}{4}N(0,1)+\tfrac{3}{4}N(6,1)$、各 $a$ で post=10000、reps=3、seed=42）だと、

* **Neff vs a**（$f(x)=x$ と $g(x)=\mathbf{1}[x<3]$ の2種類＋そのmin）
* **$\tau_{\text{int}}$ vs a**（同上）
* **accept率 vs a**
* **推定値（平均と左モード質量）vs a**
* **上位a候補の表**（$\min(N_{\text{eff},x}, N_{\text{eff},g})$ 最大化）

を一気に出力します。

---


In [ ]:

# a を連続スイープして混合効率をチェック

def target_mixture_pdf_sweep(x):
    return 0.25 * norm.pdf(x, loc=0, scale=1) + 0.75 * norm.pdf(x, loc=6, scale=1)


def run_mh_trace(step_width, n_total, target_pdf, rng):
    current_x = 0.0
    trace = np.zeros(n_total)
    accepts = 0
    for i in range(n_total):
        proposal_x = current_x + rng.normal(0, step_width)
        ratio = target_pdf(proposal_x) / target_pdf(current_x)
        if rng.random() < min(1.0, ratio):
            current_x = proposal_x
            accepts += 1
        trace[i] = current_x
    return trace, accepts / n_total


def autocorr(x, max_lag):
    x = np.asarray(x)
    x = x - x.mean()
    denom = np.dot(x, x)
    if denom == 0:
        return np.zeros(max_lag + 1)
    acf = [1.0]
    for h in range(1, max_lag + 1):
        acf.append(np.dot(x[:-h], x[h:]) / denom)
    return np.array(acf)


def tau_int(x, max_lag=200):
    acf = autocorr(x, max_lag)
    pos = acf[1:]
    pos = pos[pos > 0]
    return 1 + 2 * pos.sum()


def summarize_for_a(step_width, n_post, burn, reps, seed_base):
    neff_x_list = []
    neff_g_list = []
    tau_x_list = []
    tau_g_list = []
    accept_list = []
    mean_list = []
    pleft_list = []
    for r in range(reps):
        rng = np.random.default_rng(seed_base + r)
        trace, acc = run_mh_trace(step_width, n_post + burn, target_mixture_pdf_sweep, rng)
        trace = trace[burn:]
        gx = (trace < 3).astype(float)
        tau_x = tau_int(trace)
        tau_g = tau_int(gx)
        neff_x = n_post / tau_x if tau_x > 0 else 0.0
        neff_g = n_post / tau_g if tau_g > 0 else 0.0
        neff_x_list.append(neff_x)
        neff_g_list.append(neff_g)
        tau_x_list.append(tau_x)
        tau_g_list.append(tau_g)
        accept_list.append(acc)
        mean_list.append(trace.mean())
        pleft_list.append(gx.mean())
    return {
        "neff_x": float(np.mean(neff_x_list)),
        "neff_g": float(np.mean(neff_g_list)),
        "tau_x": float(np.mean(tau_x_list)),
        "tau_g": float(np.mean(tau_g_list)),
        "accept": float(np.mean(accept_list)),
        "mean": float(np.mean(mean_list)),
        "pleft": float(np.mean(pleft_list)),
    }


n_post = 10000
burn = 2000
reps = 3
seed_base = 42

step_grid = np.logspace(-1.2, 1.2, 25)

results = []
for a in step_grid:
    stats = summarize_for_a(a, n_post=n_post, burn=burn, reps=reps, seed_base=seed_base)
    stats["a"] = a
    stats["neff_min"] = min(stats["neff_x"], stats["neff_g"])
    results.append(stats)

best = max(results, key=lambda d: d["neff_min"])

# 可視化
fig, axes = plt.subplots(2, 2, figsize=(12, 9))

# Neff
axes[0, 0].plot(step_grid, [r["neff_x"] for r in results], label='Neff for f(x)=x')
axes[0, 0].plot(step_grid, [r["neff_g"] for r in results], label='Neff for g(x)=1[x<3]')
axes[0, 0].plot(step_grid, [r["neff_min"] for r in results], label='min(Neff_x, Neff_g)')
axes[0, 0].axvline(best["a"], color='tab:blue', ls='--', lw=1)
axes[0, 0].set_xscale('log')
axes[0, 0].set_title('MCMC効率 vs 歩幅 a (Neffが高いほど良い)')
axes[0, 0].set_xlabel('a (log scale)')
axes[0, 0].set_ylabel('N_eff (approx)')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].legend()

# tau_int
axes[0, 1].plot(step_grid, [r["tau_x"] for r in results], label='tau_int for f(x)=x')
axes[0, 1].plot(step_grid, [r["tau_g"] for r in results], label='tau_int for g(x)=1[x<3]')
axes[0, 1].axvline(best["a"], color='tab:blue', ls='--', lw=1)
axes[0, 1].set_xscale('log')
axes[0, 1].set_yscale('log')
axes[0, 1].set_title('自己相関時間 $\tau_{\text{int}}$ vs 歩幅 a (低いほど良い)')
axes[0, 1].set_xlabel('a (log scale)')
axes[0, 1].set_ylabel('tau_int (log scale)')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].legend()

# acceptance
axes[1, 0].plot(step_grid, [r["accept"] for r in results])
axes[1, 0].axvline(best["a"], color='tab:blue', ls='--', lw=1)
axes[1, 0].set_xscale('log')
axes[1, 0].set_title('受理率 (Acceptance Rate) vs 歩幅 a')
axes[1, 0].set_xlabel('a (log scale)')
axes[1, 0].set_ylabel('accept_rate')
axes[1, 0].grid(True, alpha=0.3)

# 推定値
axes[1, 1].plot(step_grid, [r["mean"] for r in results], label='mean estimate E[X]')
axes[1, 1].plot(step_grid, [r["pleft"] for r in results], label='P(X<3) estimate')
axes[1, 1].axvline(best["a"], color='tab:blue', ls='--', lw=1)
axes[1, 1].axhline(4.5, color='tab:blue', ls='--', lw=1)
axes[1, 1].axhline(0.25, color='tab:orange', ls='--', lw=1)
axes[1, 1].set_xscale('log')
axes[1, 1].set_title('推定値 vs 歩幅 a (N_postは共通)')
axes[1, 1].set_xlabel('a (log scale)')
axes[1, 1].set_ylabel('estimate')
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].legend()

plt.tight_layout()
plt.show()

# 上位候補を表示
sorted_results = sorted(results, key=lambda d: d["neff_min"], reverse=True)[:5]
print("top a candidates (by min Neff):")
for r in sorted_results:
    print(f"a={r['a']:.3f} | Neff_x={r['neff_x']:.1f} | Neff_g={r['neff_g']:.1f} | min={r['neff_min']:.1f} | accept={r['accept']:.3f}")
print(f"\\nrobust best a (min Neff): a={best['a']:.3f}")


### グラフの読み方（重要ポイントだけ）

1. **Neff曲線**
   * 小さい $a$ で Neff が死ぬのは「一歩が小さすぎて山間移動できない」＝探索が遅い
   * $a$ を増やすと Neff が急上昇する区間があり、ここが混合改善ゾーン
   * 大きすぎると reject が増えて Neff がまた落ちる

2. **$\tau_{\text{int}}$ 曲線**
   * $\tau_{\text{int}}$ は小さいほど良い
   * Neff と裏表（$N_{\text{eff}}=N/\tau_{\text{int}}$）なので最適付近で最小化される

3. **accept率曲線**
   * $a$ が増えるほど accept は下がる（当然）
   * ただし accept が高い＝良い、ではない

---

### もし「スパイクっぽい変なNeff」が出たら

$g(x)=\mathbf{1}[x<3]$ は **0/1で分散が小さくなりやすい**ため、短い系列だとACF推定が荒れてスパイクが出がちです。

**安定させるなら**

* `reps=10` に上げる
* `n_post` を増やす（3万〜10万）

のどちらかが効きます。

---

### 次の一手（推し）

理解をさらに固定するなら、同じループで

* **スイッチ回数**：$\sum_t \mathbf{1}[g_t\neq g_{t-1}]$
* **MSD**（平均二乗変位）

も同時に出すと「探索」感が一気に体感化できます。



---
## （付録）平均の分散に関する理論的考察

まず平均を
$$
\bar{y}_T = \frac{1}{T}\sum_{t=1}^T y_t
$$
と置くと、自己共分散 $\gamma_h=\mathrm{Cov}(y_t,y_{t-h})$ を用いて
$$
\mathrm{Var}(\bar{y}_T)= \frac{1}{T^2}\sum_{t=1}^T\sum_{s=1}^T\gamma_{|t-s|}
$$
となります。AR(1)では $\gamma_h=\sigma_u^2\phi^{|h|}$ なので、二重和を整理すると
$$
\mathrm{Var}(\bar{y}_T)= \frac{1}{T^2}\Big(T\gamma_0 + 2\sum_{h=1}^{T-1}(T-h)\gamma_h\Big)
$$
$$
= \frac{1}{T^2}\Big(T\sigma_u^2 + 2\sum_{h=1}^{T-1}(T-h)\sigma_u^2\phi^h\Big)
$$
となり、相関がある（$\phi \neq 0$）と、独立な場合（iid）に比べて分散が変化することがわかります。

---
